In [2]:
# !pip install soynlp
# !pip show soynlp

    100% |████████████████████████████████| 1.5MB 3.3MB/s ta 0:00:01
Name: soynlp
Version: 0.0.46
Summary: Unsupervised Korean Natural Language Processing Toolkits
Home-page: https://github.com/lovit/soynlp
Author: Lovit
Author-email: soy.lovit@gmail.com
License: UNKNOWN
Location: /anaconda3/lib/python3.6/site-packages
Requires: numpy, psutil
Required-by: 


In [2]:
import pandas as pd 
import numpy as np
import re
from soynlp.tokenizer import RegexTokenizer
import gensim
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
from gensim import corpora, models
from gensim.models import LdaMulticore
from gensim.models import Word2Vec
from gensim.corpora import Dictionary, MmCorpus

In [0]:
# 참고자료 
# http://nbviewer.jupyter.org/github/KMJJ1/hiphop/blob/master/RNN%20-%20LSTM%EC%9D%84%20%EC%9D%B4%EC%9A%A9%ED%95%9C%20Hiphop%20%EA%B0%80%EC%82%AC%20%EC%83%9D%EC%84%B1.ipynb
# https://github.com/dyelax/encore.ai

In [4]:
%pwd

'/Users/lab/Desktop/Word2Vec_Idol'

In [4]:
data= pd.read_csv("data/song_tidy03.csv")

In [5]:
song = pd.DataFrame(data)

In [6]:
song.head(3)

,title,artist,album,release_date,song_genre,is_title,like,creator,lyrics,lyricist,composer,arranger
0,latata,(여자)아이들,I am,2018.05.02,Dance,타이틀 곡,"85,374",소연 작사 소연 작곡 빅싼초 (Yummy Tone) 작곡 빅싼...,기나긴 너와 이 밤을<br>너와 이렇게 너와<br>기다린 너와 시간을<br>너와 이...,소연,"소연, 빅싼초 (Yummy Tone)","빅싼초 (Yummy Tone), 소연"
1,달라($$$),(여자)아이들,I am,2018.05.02,Dance,수록곡,"4,229",Le`mon 작사 소연 작사 윤종성 작곡 Le`mon 작곡...,Knock knock 들어간다<br>Soyeon이<br>머리 위에 Spotlight...,"Le`mon, 소연","윤종성, Le`mon",윤종성
2,maze,(여자)아이들,I am,2018.05.02,Dance,수록곡,"6,198",ZeroZine (MosPick) 작사 Ferdy (MosPick) 작사...,너의 깊은 그 눈빛이<br>날 데려가 저 멀리<br>누군가 맘에 들 때<br>더 신...,"ZeroZine (MosPick), Ferdy (MosPick), 소연","ZeroZine (MosPick), Ferdy (MosPick)","ZeroZine (MosPick), Ferdy (MosPick)"


In [7]:
song.columns

Index(['title', 'artist', 'album', 'release_date', 'song_genre', 'is_title',
       'like', 'creator', 'lyrics', 'lyricist', 'composer', 'arranger'],
      dtype='object')

In [8]:
song['artist'].value_counts().head()

쿨              226
슈퍼주니어          212
신화             206
플라이 투 더 스카이    164
동방신기           163
Name: artist, dtype: int64

In [9]:
# 유니크한 가수의 수 
len(song['artist'].unique())

328

In [10]:
# 곡의 수 
song.shape

(9582, 12)

In [11]:
song['lyrics'].isnull().sum()

0

In [12]:
song['lyrics'].head()

0    기나긴 너와 이 밤을<br>너와 이렇게 너와<br>기다린 너와 시간을<br>너와 이...
1    Knock knock 들어간다<br>Soyeon이<br>머리 위에 Spotlight...
2    너의 깊은 그 눈빛이<br>날 데려가 저 멀리<br>누군가 맘에 들 때<br>더 신...
3    오늘따라 이상하리만큼<br>잠이 오지 않아 괜히 뒤척여<br>익숙한 듯 어떻게 연락...
4    What's in your house<br>What's in your house<b...
Name: lyrics, dtype: object

## 텍스트 데이터 전처리

In [16]:
def preprocessing(text):
    # 개행문자 제거
    text =  text.strip('\t\n\r')
    pattern = re.compile(r'\s+')
    text = re.sub(pattern, ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    # text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    return text

In [17]:
sample_content = song['lyrics'][100]
type(sample_content)

str

In [18]:
sample_content = preprocessing(sample_content)
sample_content[:1000]

'모든게 보여 이제야 느껴 br 얼마나 날 사랑했는지 br 너무 가까이 있어서 내눈이 br 몰랐어 얼마나 특별한지 br 어떻게 너무 늦었어 br 넌 이미 없어 br 이제 깨달아서 와보니 br 넌 벌써 떠났어 br 다신 올 수 없는곳으로 미치겠어 br 정말 너무 보고싶어서 br 이렇게하고 떠나버리면 어떻게 br 바보야 왜 니 맘대로 결정해 br 이게 날 사랑해서야 날 위한거야 br 난 너랑 있어야 산단말이야 br I said of br 헤어짐보다 더 아픈 그리움 br I said of br 이제야 느껴지는 널 잃은 이슬품 br 돌아봤어 널 찾아헤맸어 br 니가 있을만한 곳은 모두 br 꼭 만나야했어 할말이 있었어 br 니가 떠나고서 깨달은거 br 정말 특별했던거야 br 너무나 귀했던 사랑이야 br 이제 돌아보니 br 다시는 못말날 그런거야 br 너무나 힘들어서 br 끝까지 가지 못할것같다고 br 버려선 안됐어 br 우리가 너무 몰라도 너무 몰랐어 br 헤어지지 말았어야 했어 절대 br 이말을 하려 했던건데 br 드디어 너에게로 찾았는데 br 넌 이제 없어 어떡해 br  분이라도 더 우리 br 함께 있어야 했어 br 정말  분이라도 더 우리 br 함께 있어야 했어 br I said of br 헤어짐보다 더 아픈 그리움 br I said of br 이제야 느껴지는 널 잃은 br I said of br 헤어짐보다 더 아픈 그리움 br I said of br 이제야 느껴지는 널 잃은 이슬픔 br 사랑은 진짜 사랑은 br 인생에 한번이야 br 그 사랑은 다른 사랑으로 br 채울수가 없는거야 그러니 br 니가 나를 위해서 했다는일이 br 얼마나 말이 안되는 일이니 br 다른사람이 br 너의 자릴 너 대신 매꿔 br 날 행복하게 할꺼라고 br 나를 꼭 너보다 훨씬 br 좋은사람 만나게 한다고 br 누가 너보고 생각해달래 br 내가 대체 원하는게 뭔데 br 날 행복하게 하는게 뭔데 br 그걸 왜 뺏어 그게 바로 넌데 br I said of br 헤어짐보다 더 아픈 그리움 br

In [19]:
# %time을 찍어주면 해당 코드를 실행할 때 걸리는 시간을 출력해 줍니다
%time sentences = song['lyrics'].apply(preprocessing)

CPU times: user 1.51 s, sys: 13.9 ms, total: 1.52 s
Wall time: 1.53 s


In [20]:
sentences.head()

0    기나긴 너와 이 밤을 br 너와 이렇게 너와 br 기다린 너와 시간을 br 너와 이...
1    Knock knock 들어간다 br Soyeon이 br 머리 위에 Spotlight...
2    너의 깊은 그 눈빛이 br 날 데려가 저 멀리 br 누군가 맘에 들 때 br 더 신...
3    오늘따라 이상하리만큼 br 잠이 오지 않아 괜히 뒤척여 br 익숙한 듯 어떻게 연락...
4    What s in your house br What s in your house b...
Name: lyrics, dtype: object

## Bigram 만들기

'말이 되는' 가사 생성에 도움받기 위해 자주 나오는 단어들을 묶어 어구로 만들어야 한다.

n-gram 이라는 기법으로 bigram(두 단어씩 묶기), trigram(세 단어씩 묶기) 등의 방법이 있다.

`bigram = gensim.models.Phrases(tokens, min_count=1, threshold=100)`이런식으로 만들 수 있다.

trigram을 하려면 bigram 으로 나온 것을 한번 더 Phrases에 넣고 돌려주는 방식으로 하면 된다.

// to create the bigrams
bigram_model = Phrases(unigram_sentences)

// apply the trained model to a sentence
 for unigram_sentence in unigram_sentences:                
            bigram_sentence = u' '.join(bigram_model[unigram_sentence])

// get a trigram model out of the bigram
trigram_model = Phrases(bigram_sentences)

In [21]:
# 토크나이징 하기 전 sentences 로 txt파일 생성

USE_PREMADE_TEXT = False

text_filepath = 'all_lyrics_text.txt'
if not USE_PREMADE_TEXT:
    with open(text_filepath, 'w', encoding='utf-8') as f:
        for lyrics in sentences.values:
            if pd.isnull(lyrics): # null값 있다면 그 다음으로 넘어감
                continue
            f.write(lyrics + '\n')
else:
    assert path.exists(text_filepath)

In [22]:
# bigram model 저장 


USE_PREMADE_BIGRAM_MODEL = False

all_bigram_model_filepath = 'all_bigram_model'
all_sentences_normalized_filepath = 'all_lyrics_text.txt'

all_unigram_sentences = LineSentence(all_sentences_normalized_filepath)

if not USE_PREMADE_BIGRAM_MODEL:    
    
    all_bigram_model = Phrases(all_unigram_sentences) #phrase냐 아니냐를 판단해줌
    all_bigram_model.save(all_bigram_model_filepath)
    
else:
    all_bigram_model = Phrases.load(all_bigram_model_filepath)

In [24]:
%%time
USE_PREMADE_BIGRAM_SENTENCES = False

all_bigram_sentences_filepath = 'all_sentences_for_word2vec.txt'

if not USE_PREMADE_BIGRAM_SENTENCES:
    
    with open(all_bigram_sentences_filepath, 'w', encoding='utf-8') as f:
        for unigram_sentence in all_unigram_sentences:
            all_bigram_sentence = all_bigram_model[unigram_sentence]
            f.write(' '.join(all_bigram_sentence) + '\n')
else:
    assert path.exists(all_bigram_sentences_filepath)

/Users/jieun/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 11.2 s, sys: 52 ms, total: 11.2 s
Wall time: 11.4 s


## Word2vec 모델 만들기

Skip-gram 방식으로 word2vec 하였다.

Skip-Gram 방식은 CBOW 방식과 반대로 특정한 단어로부터 문맥이 될 수 있는 단어를 예측한다. 

보통 입력 단어 주변의 kk개 단어를 문맥으로 보고 예측 모형을 만드는데 이 kk 값을 window size 라고 한다.


In [25]:
# word2vec 모델 학습에 로그를 찍을 수 있도록 합니다.

import logging
logging.basicConfig(
    format='%(asctime)s : %(levelname)s : %(message)s', 
    level=logging.INFO)

In [29]:
# all2vec.corpus_count  # total_examples, 총 9582개의 가사를 학습 

9582

In [28]:
%%time
USE_PREMADE_WORD2VEC = False

all2vec_filepath = 'all_word2vec_model'

if not USE_PREMADE_WORD2VEC:
    
    lyrics_for_word2vec = LineSentence(all_bigram_sentences_filepath)

    all2vec = Word2Vec(lyrics_for_word2vec, size=50, window=5, min_count=1, sg=1)
    # sg=1 , Skip-Gram Model
    # window = 5 앞 5개, 뒤 5개 단어를 보겠다는 뜻
    # window size 작을수록 문법적인 의미가 너무 중요해짐, 클수록 주제 지향적으로 문맥적인 정보를 많이 담게 됨    
    for _ in range(9):
        all2vec.train(lyrics_for_word2vec,total_examples=9582, epochs=1)  # 총 9582'문장'을 학습
        
        
    all2vec.save(all2vec_filepath)
else:
    all2vec = Word2Vec.load(all2vec_filepath)
all2vec.init_sims()

2018-09-30 19:18:06,073 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2018-09-30 19:18:06,076 : INFO : collecting all words and their counts
2018-09-30 19:18:06,080 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-09-30 19:18:06,935 : INFO : collected 142784 word types from a corpus of 2405886 raw words and 9582 sentences
2018-09-30 19:18:06,937 : INFO : Loading a fresh vocabulary
2018-09-30 19:18:07,349 : INFO : min_count=1 retains 142784 unique words (100% of original 142784, drops 0)
2018-09-30 19:18:07,351 : INFO : min_count=1 leaves 2405886 word corpus (100% of original 2405886, drops 0)
2018-09-30 19:18:07,785 : INFO : deleting the raw counts dictionary of 142784 items
2018-09-30 19:18:07,790 : INFO : sample=0.001 downsamples 19 most-common words
2018-09-30 19:18:07,792 : INFO : downsampling leaves estimated 1841716 word corpus (76.6% of prior 2405886)
2018-09-30 19:18:08,217 : INFO : estimated required memory fo

2018-09-30 19:18:47,688 : INFO : EPOCH 1 - PROGRESS: at 17.30% examples, 310368 words/s, in_qsize 6, out_qsize 0
2018-09-30 19:18:48,689 : INFO : EPOCH 1 - PROGRESS: at 33.00% examples, 313252 words/s, in_qsize 5, out_qsize 0
2018-09-30 19:18:49,699 : INFO : EPOCH 1 - PROGRESS: at 50.07% examples, 313451 words/s, in_qsize 5, out_qsize 0
2018-09-30 19:18:50,740 : INFO : EPOCH 1 - PROGRESS: at 67.70% examples, 311800 words/s, in_qsize 5, out_qsize 0
2018-09-30 19:18:51,751 : INFO : EPOCH 1 - PROGRESS: at 85.99% examples, 313539 words/s, in_qsize 5, out_qsize 0
2018-09-30 19:18:52,475 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-30 19:18:52,529 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-30 19:18:52,536 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-30 19:18:52,538 : INFO : EPOCH - 1 : training on 2405886 raw words (1841548 effective words) took 5.9s, 313934 effective words/s
2018-09-30 19:18:52,539 :

2018-09-30 19:19:25,078 : INFO : EPOCH 1 - PROGRESS: at 46.81% examples, 292024 words/s, in_qsize 5, out_qsize 0
2018-09-30 19:19:26,088 : INFO : EPOCH 1 - PROGRESS: at 63.14% examples, 292301 words/s, in_qsize 5, out_qsize 0
2018-09-30 19:19:27,099 : INFO : EPOCH 1 - PROGRESS: at 79.45% examples, 293712 words/s, in_qsize 5, out_qsize 0
2018-09-30 19:19:28,102 : INFO : EPOCH 1 - PROGRESS: at 97.28% examples, 296003 words/s, in_qsize 5, out_qsize 0
2018-09-30 19:19:28,211 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-30 19:19:28,216 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-30 19:19:28,244 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-30 19:19:28,246 : INFO : EPOCH - 1 : training on 2405886 raw words (1841889 effective words) took 6.2s, 296474 effective words/s
2018-09-30 19:19:28,248 : INFO : training on a 2405886 raw words (1841889 effective words) took 6.2s, 296335 effective words/s
2018-09-30 

CPU times: user 4min 13s, sys: 1.88 s, total: 4min 15s
Wall time: 1min 29s


In [30]:
all2vec_filepath = 'all_word2vec_model'
all2vec = Word2Vec.load(all2vec_filepath)

2018-09-30 19:19:46,602 : INFO : loading Word2Vec object from all_word2vec_model
2018-09-30 19:19:47,405 : INFO : loading wv recursively from all_word2vec_model.wv.* with mmap=None
2018-09-30 19:19:47,407 : INFO : setting ignored attribute vectors_norm to None
2018-09-30 19:19:47,409 : INFO : loading vocabulary recursively from all_word2vec_model.vocabulary.* with mmap=None
2018-09-30 19:19:47,410 : INFO : loading trainables recursively from all_word2vec_model.trainables.* with mmap=None
2018-09-30 19:19:47,413 : INFO : setting ignored attribute cum_table to None
2018-09-30 19:19:47,414 : INFO : loaded all_word2vec_model


In [31]:
# 총 얻은 단어의 개수 

a = pd.DataFrame(all2vec.wv.index2word)
len(a)

142784

In [32]:
len(a)

142784

## 100회 이상 등장한 단어들만 가지고 t-SNE 시각화

t-SNE란 차원축소 기법의 하나로 시각화를 위해 주로 사용된다

t-SNE는 고차원에서 가까운 포인트를 저차원에서도 계속 가깝게 두고, 먼 포인트는 계속 멀게 유지시킨다

데이터간의 유사도를 통해 데이터의 패턴을 찾을 수 있다

단어 중 100회 초과 등장한 단어들을 대상으로, word2vec로 얻어진 각 단어의 numpy 행렬값을 매핑시킨 후 2차원 그래프로 시각화하였다.


In [51]:
%%time
words = []
for i in (range(len(a))):
    cnt = all2vec.wv.vocab[a[0][i]] # 횟수 카운트
    if cnt.count > 50:
        words.append(a[0][i]) # 횟수 50회 초과하는 단어만 리스트화
    i += 1

CPU times: user 1.84 s, sys: 10.4 ms, total: 1.85 s
Wall time: 1.87 s


In [52]:
len (words)  # 50회 이상 등장한 단어의 개수

4523

In [53]:
# 100회 이상 등장한 단어들에 대해 vector값 가져오기

X = []
for i in words:
    X.append(all2vec.wv[i])

In [54]:
X2 = pd.DataFrame(X, index = words)

In [55]:
# 단어와 각 vector 값으로 데이터 프레임 생성
X2.head(10)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
br,0.177568,-0.199902,0.066207,0.236539,0.156750,-0.055757,0.010011,0.147721,-0.258792,0.146014,...,0.371643,-0.081813,0.146290,-0.118242,-0.149471,0.186198,-0.288082,0.048588,-0.082450,0.444372
난,0.506615,0.127678,0.371831,0.124418,0.469852,-0.101051,0.342208,0.169653,-0.239423,-0.206196,...,0.270636,-0.577222,0.394851,-0.261188,-0.669099,0.161227,-0.650018,0.185608,0.038491,0.727613
내,0.159916,-0.552918,0.216852,-0.072673,0.133223,-0.282433,0.362362,-0.106217,-0.014937,-0.343718,...,-0.080101,-0.485248,0.020429,-0.092978,-0.337727,0.214596,-1.004609,-0.126122,-0.225668,0.342476
you,0.570039,-0.579843,0.509513,0.970117,0.235792,-0.653044,-0.082132,0.711692,-0.357223,0.002319,...,1.280778,-0.262877,0.114713,-0.609323,0.011034,0.065404,-0.152555,-0.060185,-0.510247,0.341287
날,0.145147,-0.797586,0.023313,0.363200,0.108915,0.339257,-0.024092,0.368439,-0.660227,-0.072114,...,0.025784,-0.580138,-0.096489,0.158263,-0.274105,0.597724,-0.360257,-0.281170,-0.266563,0.828153
널,-0.005605,-0.582876,-0.365176,0.334459,0.366966,-0.259628,-0.096996,0.994664,-0.181762,-0.214953,...,0.487798,-0.478981,0.193865,-0.104443,-0.084474,0.691072,-0.747010,-0.038576,0.563843,0.903108
내가,-0.144855,-0.436843,0.128285,-0.111689,-0.078834,0.366055,-0.104277,0.221818,-0.097596,0.353047,...,0.617147,-0.905975,-0.241239,-0.047820,-0.647710,-0.181564,-0.682482,0.201615,-0.121637,0.681562
나,0.212652,-0.416647,-0.028169,-0.535292,0.017181,0.172259,0.203645,0.689763,-0.102919,-0.190638,...,0.277096,-0.745738,0.067175,-0.631599,-0.485521,-0.232026,-0.670722,0.224940,-0.252053,0.524468
넌,-0.017817,-0.403012,0.620517,0.593321,0.517718,-0.291012,-0.171562,0.484168,-0.239843,0.120770,...,0.091589,-0.336456,0.660529,0.554664,-0.458508,-0.139940,-0.544133,-0.163985,0.416451,0.166817
I,0.613479,-0.918006,0.348711,1.033189,0.023338,-0.128095,-0.241453,0.102001,-0.498362,0.188850,...,0.504094,-0.192065,0.421881,-1.050605,0.034282,-0.170279,-0.383492,-0.555073,-0.277010,-0.154144


In [56]:
import pickle

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE

# t-SNE 모델 모델링 

USE_PREMADE_TSNE = False

tsne_filepath = 'tsne.pkl'

if not USE_PREMADE_TSNE:
    
    tsne = TSNE(random_state=0)
    tsne_points = tsne.fit_transform(X2)
    with open(tsne_filepath, 'wb') as f:
        pickle.dump(tsne_points, f)
else:
    with open(tsne_filepath, 'rb') as f:
        tsne_points = pickle.load(f)

tsne_df = pd.DataFrame(tsne_points, index=X2.index, columns=['x_coord', 'y_coord'])
tsne_df['word'] = tsne_df.index

In [57]:
# t-SNE 모델 시각화

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value
output_notebook()

# prepare the data in a form suitable for bokeh.
plot_data = ColumnDataSource(tsne_df)

# create the plot and configure it
tsne_plot = figure(title='t-SNE Word Embeddings',
                   plot_width = 800,
                   plot_height = 800,
                   active_scroll='wheel_zoom'
                  )

# add a hover tool to display words on roll-over
tsne_plot.add_tools( HoverTool(tooltips = '@word') )

tsne_plot.circle('x_coord', 'y_coord', source=plot_data,
                 color='blue', line_alpha=0.2, fill_alpha=0.1,
                 size=10, hover_line_color='orange')

# adjust visual elements of the plot
tsne_plot.title.text_font_size = value('16pt')
tsne_plot.xaxis.visible = False
tsne_plot.yaxis.visible = False
tsne_plot.grid.grid_line_color = None
tsne_plot.outline_line_color = None

# show 
show(tsne_plot);

Loading BokehJS ...

In [71]:
from bokeh.plotting import figure
from bokeh.resources import CDN
from bokeh.embed import file_html


html = file_html(tsne_plot, CDN, "my plot")

In [73]:
html  # 이걸 홈페이지에 임베딩 시킬 수 있을지 고민해보기 

'\n<!DOCTYPE html>\n<html lang="en">\n    <head>\n        <meta charset="utf-8">\n        <title>my plot</title>\n        \n<link rel="stylesheet" href="https://cdn.pydata.org/bokeh/release/bokeh-0.12.10.min.css" type="text/css" />\n        \n<script type="text/javascript" src="https://cdn.pydata.org/bokeh/release/bokeh-0.12.10.min.js"></script>\n<script type="text/javascript">\n    Bokeh.set_log_level("info");\n</script>\n        <style>\n          html {\n            width: 100%;\n            height: 100%;\n          }\n          body {\n            width: 90%;\n            height: 100%;\n            margin: auto;\n          }\n        </style>\n    </head>\n    <body>\n        \n        <div class="bk-root">\n            <div class="bk-plotdiv" id="3132b02c-d416-4883-9e64-9d186da94d25"></div>\n        </div>\n        \n        <script type="text/javascript">\n            (function() {\n          var fn = function() {\n            Bokeh.safely(function() {\n              (function(ro

각 데이터 포인트에 마우스를 올리면 어떤 단어들이 함께 맵핑되어 있는지 확인 가능하다. 

- 진한 부분일 수록 비슷한 단어들이 많이 모여 있음을 알 수 있다.


- 한글, 영어가 끼리끼리 모여서 임베딩되었다. 



## 임베딩 연산 확인해보기 

In [148]:
# (여자, 왕) - 남자 = 여왕 
# 사랑 : 남자 = 사랑인걸,나의_첫사랑,내사랑 : 여자 
all2vec.most_similar(positive=['여자', '사랑'], negative=['남자'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('하고_싶어요', 0.6534351110458374),
 ('사랑인걸', 0.6457100510597229),
 ('전부였을때', 0.6389805674552917),
 ('나의_첫사랑', 0.6388945579528809),
 ('이해해', 0.6380597352981567),
 ('세상이_우릴', 0.634536862373352),
 ('내사랑', 0.6315751671791077),
 ('서툴렀던', 0.6283363103866577),
 ('됐지만', 0.6275029182434082),
 ('상처가', 0.6243001818656921)]

In [149]:
# 사랑 : 여자 = 사람아,눈물만큼,내사랑,나의_바램을 : 남자
all2vec.most_similar(positive=['남자', '사랑'], negative=['여자'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('사람아', 0.6498801112174988),
 ('눈물만큼', 0.6185471415519714),
 ('내사랑', 0.6168158054351807),
 ('나의_바램을', 0.6143080592155457),
 ('사랑틀', 0.6121094822883606),
 ('믿기는', 0.6098517775535583),
 ('할수있는', 0.608017086982727),
 ('이별', 0.5930275917053223),
 ('사랑할게요', 0.5912532806396484),
 ('사랑속', 0.5910249948501587)]

In [150]:
# 사랑 - 남자 = 아픔, 눈물속에, 그리움이, 상처가
all2vec.most_similar(positive=['사랑'], negative=['남자'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('아픔이', 0.5382543802261353),
 ('깊이', 0.5124201774597168),
 ('눈물속에', 0.5119723081588745),
 ('그리움이', 0.49767443537712097),
 ('상처가', 0.485943466424942),
 ('말하지_못했던', 0.48543861508369446),
 ('끝나지_않아', 0.47092998027801514),
 ('끝나지_않은', 0.46276113390922546),
 ('우철', 0.457460880279541),
 ('후회', 0.4541429281234741)]

In [151]:
# 사랑 - 여자 = 끝나지 않을, 버리고_가, 떠나가네 
all2vec.most_similar(positive=['사랑'], negative=['여자'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('끝나지_않은', 0.5181803107261658),
 ('버리고_가', 0.5167304873466492),
 ('떠나가네', 0.503881573677063),
 ('너의_흔적', 0.5022042989730835),
 ('눈물속에', 0.49522578716278076),
 ('끝나지_않아', 0.4951077401638031),
 ('후회', 0.4749501645565033),
 ('않기를', 0.4738181233406067),
 ('눈물만큼', 0.4686936140060425),
 ('가슴에_묻어', 0.4642975926399231)]

In [108]:
all2vec.n_similarity(['사랑'], ['돈'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.15197162705746212

In [144]:
all2vec.n_similarity(['여자'], ['남자'])  # 여자와 남자는 가사속에서는 엄청 유사하다 

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.7327100211536973

In [153]:
all2vec.n_similarity(['여자'], ['오빠'])  # 여자는 오빠랑 더 유사하다. 

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.5467148473519785

In [154]:
all2vec.n_similarity(['여자'], ['누나'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.4290209398449761

In [156]:
all2vec.n_similarity(['남자'], ['오빠'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.45690119269148843

In [155]:
all2vec.n_similarity(['남자'], ['누나'])  # 남자는 누나랑 더 유사하다 

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.5004946247256581

## 단어간의 유사도보기 : (주의) 가사로 학습을 했기 때문에 ...
[여자] : [x]의 유사도를 구하면 이상한 결과가 나온다. 따라서 노래가사말 같은(구어체) 단어인 
[여자는, 여잔, 남자는, 남잔] 으로 비교해보는게 더욱 정확하다.

In [166]:
all2vec.n_similarity(['여자는'], ['외모'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.5145306581663699

In [167]:
all2vec.n_similarity(['남자는'], ['외모'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.4194518740901746

In [260]:
all2vec.n_similarity(['여잔'], ['외모'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.5649161601817115

In [261]:
all2vec.n_similarity(['남잔'], ['외모'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.4524207378113294

In [182]:
all2vec.n_similarity(['여자는'], ['키'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.4326799603786505

In [184]:
all2vec.n_similarity(['남자는'], ['키'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.4922769825893118

In [258]:
all2vec.n_similarity(['여잔'], ['키'])  # 이게 왜 더 높은건지는 잘 모르겠다. 여잔 키가 작아도 괜찮아일까? 키는 상관없어 일까? 

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.5043553757554659

In [259]:
all2vec.n_similarity(['남잔'], ['키'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.40955324378709324

In [180]:
all2vec.n_similarity(['여자는'], ['얼굴'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.3934655952428614

In [181]:
all2vec.n_similarity(['남자는'], ['얼굴'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.4178795669397087

In [256]:
all2vec.n_similarity(['여잔'], ['얼굴'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.365155187102501

In [257]:
all2vec.n_similarity(['남잔'], ['얼굴'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.2987883439166516

In [186]:
all2vec.n_similarity(['여자는'], ['몸매'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.48054625754337643

In [179]:
all2vec.n_similarity(['남자는'], ['몸매'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.41677229346642997

In [254]:
all2vec.n_similarity(['여잔'], ['몸매'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.516577467501453

In [255]:
all2vec.n_similarity(['남잔'], ['몸매'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.4340661422457258

In [189]:
all2vec.n_similarity(['여자는'], ['성격'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.4059434403575314

In [188]:
all2vec.n_similarity(['남자는'], ['성격'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.4445830344495734

In [198]:
all2vec.n_similarity(['여잔'], ['성격'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.5642451030658595

In [199]:
all2vec.n_similarity(['남잔'], ['성격'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.5475935137298052

In [266]:
all2vec.n_similarity(['누나'], ['능력'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.2477376679071554

In [267]:
all2vec.n_similarity(['오빠'], ['능력'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.3270536940871518

In [268]:
all2vec.n_similarity(['여자'], ['능력'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.3265228307434888

In [270]:
all2vec.n_similarity(['남자'], ['능력'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.3952032055475089

In [200]:
all2vec.n_similarity(['누나'], ['몸매'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.42132763277128316

In [201]:
all2vec.n_similarity(['오빠'], ['몸매'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.3527413000329873

In [202]:
all2vec.n_similarity(['누나'], ['얼굴'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.4151277876947488

In [203]:
all2vec.n_similarity(['오빠'], ['얼굴'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.26806970426593507

데잇걸즈 여러분의 아이디어를 받아서 실험해본 결과! 

~ 해줘 하는 청유형 문장이 왠지 여자아이돌 노래에서 더 자주 나올 것 같아요!
- 수동적/능동적 태도 차이(~해줘/~할게, 기다릴게/갈게, 숨길게/뺏을게 등?)
- "오빠"/"누나" 횟수와 함께 "너" 의 횟수도 보면 재밌을 것 같아요!
-이성의 신체 부위 관련 단어 등장 빈도요! (남자가수 : 머리카락, 머릿결, 눈, 입술, 엉덩이, 허리, 다리, 몸매 등 / 여자가수 : 눈빛,  어깨…흠 딱히 더 생각나지 않지만 있다면 추가해서…)
- 의상을 가리키는 단어! (비키니, 청바지, 치마, 교복, 수트 등등)
- 그리고 존댓말/반말 차이도 궁금해요!

In [214]:
all2vec.n_similarity(['오빠'], ['너'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.14214628453155262

In [215]:
all2vec.n_similarity(['누나'], ['너'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.


0.35538761249063766

## 가장 비슷한 단어 보기 

In [163]:
all2vec.most_similar(positive=['누나'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('말하기', 0.8114107251167297),
 ('애들아', 0.7541360259056091),
 ('누날', 0.7428486347198486),
 ('이것보다', 0.7307059168815613),
 ('뭣하러', 0.7250329256057739),
 ('동생', 0.7219783663749695),
 ('몇살', 0.7206633687019348),
 ('옴마야', 0.7131503224372864),
 ('축하합니다', 0.712969958782196),
 ('나와가지고', 0.7126736044883728)]

In [160]:
all2vec.most_similar(positive=['오빠'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('동생', 0.7443451881408691),
 ('스타일에', 0.7334033846855164),
 ('좋으면', 0.7287778854370117),
 ('국제적인', 0.7266085743904114),
 ('떴다_하면', 0.706525981426239),
 ('대로_할게', 0.7001639008522034),
 ('이상형에', 0.6992389559745789),
 ('당돌해', 0.6885830163955688),
 ('걔랑', 0.6866766214370728),
 ('대로_시키는', 0.686146080493927)]

In [216]:
all2vec.most_similar(positive=['비키니'])  # 수영복에, 선글라스는, 자동차, 괌, 생각보다 sexual한 단어는 없음

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('수영복에', 0.9313409924507141),
 ('바하마', 0.8810994029045105),
 ('선글라스는', 0.8731160163879395),
 ('자동차', 0.8706044554710388),
 ('괌', 0.857925534248352),
 ('땀으로', 0.8493465781211853),
 ('썬그라스', 0.8483404517173767),
 ('노랑', 0.8482531905174255),
 ('뻣뻣한', 0.8477893471717834),
 ('요트를', 0.8425098657608032)]

In [232]:
all2vec.most_similar(positive=['교복'])  # 단발머리, 하이힐로, 치만, 여성 지칭단어가 많음

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('단발머리', 0.8226224184036255),
 ('하이힐로', 0.8205510377883911),
 ('치만', 0.8038955926895142),
 ('자서', 0.7817257642745972),
 ('반바지', 0.7773746252059937),
 ('도둑마냥', 0.7705428600311279),
 ('어떤걸까', 0.7696269154548645),
 ('헤매기에', 0.7656112909317017),
 ('타나', 0.7654300928115845),
 ('콧소리', 0.7624959945678711)]

In [224]:
all2vec.most_similar(positive=['넥타이'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('아에이오우', 0.9064053297042847),
 ('채워드릴게', 0.880197286605835),
 ('여친도', 0.8758857846260071),
 ('팍팍', 0.8757866621017456),
 ('우리다워', 0.8657227754592896),
 ('진땀은', 0.8629465103149414),
 ('깨버려', 0.8627868890762329),
 ('놈들보다', 0.8623360991477966),
 ('정지돼', 0.861864447593689),
 ('와리가리', 0.861790657043457)]

In [222]:
all2vec.most_similar(positive=['치마'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('높은_하이힐', 0.7932949662208557),
 ('청순해', 0.7793439030647278),
 ('and_white', 0.7792421579360962),
 ('향수도', 0.770771861076355),
 ('막히게', 0.7649984955787659),
 ('포즈를', 0.762865424156189),
 ('거리보다', 0.7607216238975525),
 ('얼음을', 0.7605347037315369),
 ('야릇해', 0.7589484453201294),
 ('volume은', 0.7560955286026001)]

In [70]:
all2vec.most_similar(positive=['사랑해'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('널_사랑해', 0.8358373641967773),
 ('너만을_사랑해', 0.7596602439880371),
 ('란말', 0.7572644352912903),
 ('어색하지만', 0.7504624724388123),
 ('사랑해_사랑해', 0.746197521686554),
 ('슬퍼마', 0.7380000352859497),
 ('우리사랑하다가', 0.7378554940223694),
 ('밝은빛깔', 0.7366132140159607),
 ('우리우리우리', 0.7352828979492188),
 ('죽는_날까지', 0.7336909770965576)]

In [164]:
all2vec.most_similar(positive=['여자는'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('여자를', 0.747633159160614),
 ('여자들은', 0.7179417610168457),
 ('착하고', 0.7079247832298279),
 ('쪽을', 0.7017565965652466),
 ('너_하나밖에', 0.696109414100647),
 ('한명일', 0.6957125663757324),
 ('듣질_않아', 0.6936326026916504),
 ('척만', 0.6868671774864197),
 ('여잔', 0.6854044198989868),
 ('걔는', 0.6800103187561035)]

In [165]:
all2vec.most_similar(positive=['남자는'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('티도', 0.7184788584709167),
 ('애가', 0.7124828696250916),
 ('바보야', 0.711473286151886),
 ('무시하잖아', 0.7058546543121338),
 ('남잔', 0.7054417133331299),
 ('너같이', 0.7036365270614624),
 ('남자', 0.701921284198761),
 ('나쁜_남자', 0.6998971104621887),
 ('사람은', 0.6974261403083801),
 ('어리숙해', 0.697398841381073)]

In [271]:
all2vec.most_similar(positive=['girl'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('Girl', 0.7749955058097839),
 ('gir_gir', 0.7706440091133118),
 ('Tough', 0.7657442688941956),
 ('sassy', 0.7637359499931335),
 ('lady', 0.7555986642837524),
 ('gir', 0.7481988668441772),
 ('Maxican', 0.7464033961296082),
 ('lady_lady', 0.743831217288971),
 ('Im_a', 0.7431143522262573),
 ('pretty_girl', 0.7418482303619385)]

In [40]:
all2vec.most_similar(positive=['boy'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('girl', 0.7346512675285339),
 ('손_놓지마', 0.7262905240058899),
 ('nah', 0.7261207103729248),
 ('little', 0.7213090658187866),
 ('mamama', 0.720516562461853),
 ('So_Sexy', 0.7170931100845337),
 ('i', 0.7148666977882385),
 ('handsome', 0.7140861749649048),
 ('비즈니스처럼', 0.7132685780525208),
 ('멀었잖니', 0.7118117213249207)]

In [42]:
all2vec.most_similar(positive=['몸매'])

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('잘록한', 0.7779519557952881),
 ('뽐내', 0.7686123847961426),
 ('거리로_나가', 0.7664014101028442),
 ('썬그라스', 0.7518778443336487),
 ('포스', 0.7485241293907166),
 ('맛의', 0.7477290630340576),
 ('청순해', 0.7455165982246399),
 ('잘빠진', 0.7421353459358215),
 ('몸매는', 0.738463819026947),
 ('탄력있는', 0.7374888062477112)]

In [245]:
all2vec.most_similar(positive=['이별'])  # ...이별은 불치병..넌 쉽니? 

/Users/jieun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('불치병', 0.7159072160720825),
 ('넌_쉽니', 0.6992561221122742),
 ('줄_모르고', 0.697421669960022),
 ('바랬는데', 0.6907091736793518),
 ('지나가겠지', 0.6875361800193787),
 ('사랑', 0.6838216781616211),
 ('이루어질_수', 0.6815088987350464),
 ('갖지_못할', 0.6804484128952026),
 ('별것', 0.6778597235679626),
 ('수집가이죠', 0.6705273985862732)]